In [ ]:

import os
import math
from statistics import mean 

from tqdm.notebook import tqdm


from PIL import Image
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.utils import shuffle

import tensorflow as tf
from keras import Input
from keras.layers import Dense, Reshape, LeakyReLU, Conv2D, Conv2DTranspose, Flatten, Dropout, Concatenate, BatchNormalization
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop



In [ ]:
def read_data(image_dir, width, height, attribute_data, n_images):
    images = []
    attributes = []
    for i in tqdm(range(n_images)):
        image_id, current = attribute_data.iloc[i, 0], attribute_data.iloc[i, 1:]
        current[current < 0] = 0
        crop = (30, 55, 150, 175) #croping size for the image so that only the face at centre is obtained
        image = np.array((Image.open(image_dir+image_id).crop(crop)).resize((width, height)))

#
        image = ((image - image.min())/(255 - image.min()))
#         image = cv2.imread(image_dir+image_id)
#         image = image[:, :, [2, 1, 0]]
#         image = image[55:175, 30:150]
#         image = cv2.resize(image, (width, height))
        images.append(image)
        attributes.append(current)

#     for i in range(len(images)):
#         images[i] = ((images[i] - images[i].min())/(255 - images[i].min()))
#         #images[i] = images[i]*2-1  #uncomment this if activation is tanh for generator last layer
    
    images = np.array(images)
    attributes = np.array(attributes)
    return images, attributes


In [ ]:
def show_images(images):
    size = min(int(math.sqrt(len(images))), 5)
    plt.figure(1, figsize=(10, 10))
    for i in range(size**2):
        plt.subplot(size, size, i+1)
        plt.imshow(images[i])
        plt.axis('off')
    plt.show()


In [ ]:
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = np.random.randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = np.array(images[ix],dtype='float32'), np.array(labels[ix], dtype='float32')
    # generate class labels
    y = np.ones((n_samples, 1))
    return [X, labels], y

In [ ]:
def generate_latent_points(latent_dim, n_samples, n_attributes):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = []
    for _ in range(n_samples):
        labels.append(np.random.randint(1, 2, n_attributes))
    z_input, labels = np.array(z_input), np.array(labels, dtype=np.float32)
    return [z_input, labels]

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples, n_attributes):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples, n_attributes)
	# predict outputs
	images = generator.predict(z_input)
	# create class labels
	y = np.zeros((n_samples, 1))
	return [images, labels_input], y

In [ ]:
def get_generator(latent_dim = 100, channels = 3, n_attributes=2):
    gen_input = Input(shape=(latent_dim, ))
    #100,1
    
#     label_input = Input(shape=(n_attributes,))
#     label_intermediate = Dense(4*4)(label_input)
#     label_reshaped = Reshape((4,4,1))(label_intermediate)

    x = Dense(512 * 4 * 4)(gen_input)
    x = Reshape((4, 4, 512))(x)
    
#     merge = Concatenate()([x, label_reshaped])

        
    x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    #16 * 16 * 256

    x = Conv2DTranspose(128, 4, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    #32*32*256

    x = Conv2DTranspose(64, 4, strides=2, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    #64*64*256

    output = Conv2DTranspose(channels, 4, strides=2, padding='same', activation='sigmoid')(x)
    #128*128*256

#     x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
#     x = LeakyReLU()(x)
    #256*256*256

#     x = Conv2D(512, kernel_size=(3, 3), padding='same')(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU()(x)
#     #256*256*512

#     output = Conv2D(channels, 4, strides=2, padding='same', activation='sigmoid')(x)
    #256*256*512

#     output = Conv2D(channels, kernel_size=(3, 3), activation='relu', padding='same')(x)
    #256*256*3

    generator = Model(gen_input, output)

    return generator


In [ ]:
def get_discriminator(height, width, channels, n_attributes=40):
    disc_input = Input(shape=(height, width, channels))

#     label_input = Input(shape=(n_attributes,))
    
#     n_nodes = height * width
#     label_intermediate = Dense(n_nodes, activation='relu')(label_input)
#     label_intermediate = Reshape((height,width,1))(label_intermediate)
#     merge = Concatenate()([disc_input, label_intermediate])
    
    x = Conv2D(32, 4, strides=2, padding='same')(disc_input)
   
    x = Conv2D(64, 4, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(128, 4, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256, 4, strides=2, padding='same')(x)
    x = LeakyReLU(0.2)(x)
    
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(1, activation='sigmoid')(x)
    discriminator = Model(disc_input, x)
    
    
    
    return discriminator


In [ ]:
IMAGE_DIRECTORY = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'
TOTAL_IMAGES = len(os.listdir(IMAGE_DIRECTORY))
print(TOTAL_IMAGES)

WIDTH, HEIGHT = 128,128
LATENT_DIM = 100

attribute_data = pd.read_csv('../input/celeba-dataset/list_attr_celeba.csv')
attribute_data = attribute_data[['image_id', 'Male', 'Young']]

In [ ]:
noise_shape = 100

In [ ]:
gen_input = Input(shape=(100, ))

x = Dense(128 * 16 * 16)(gen_input)
x = LeakyReLU()(x)
x = Reshape((16, 16, 128))(x)

x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2D(512, 5, padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(512, 5, padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(3, 7, activation='tanh', padding='same')(x)

generator = Model(gen_input, x)

In [ ]:
disc_input = Input(shape=(128, 128, 3))

x = Conv2D(256, 3)(disc_input)
x = LeakyReLU()(x)

x = Conv2D(256, 4, strides=2)(x)
x = LeakyReLU()(x)

x = Conv2D(256, 4, strides=2)(x)
x = LeakyReLU()(x)

x = Conv2D(256, 4, strides=2)(x)
x = LeakyReLU()(x)

x = Conv2D(256, 4, strides=2)(x)
x = LeakyReLU()(x)

x = Flatten()(x)
x = Dropout(0.4)(x)

x = Dense(1, activation='sigmoid')(x)
discriminator = Model(disc_input, x)

In [ ]:
 optimizer = RMSprop(
        lr=.0001,
        clipvalue=1.0,
        decay=1e-8
    )
discriminator.compile(
        optimizer=optimizer,
        loss='binary_crossentropy'
    )
gan_input = Input(shape=(100, ))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

optimizer = RMSprop(lr=.0001, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=optimizer, loss='binary_crossentropy')

In [ ]:
# gan =Sequential([generator,discriminator])
# discriminator.compile(optimizer='adam',loss='binary_crossentropy')
# discriminator.trainable = False
# gan.compile(optimizer='adam',loss='binary_crossentropy')


In [ ]:
tf.keras.utils.plot_model(gan, show_shapes=True)

In [ ]:
images, attributes = read_data(IMAGE_DIRECTORY, WIDTH, HEIGHT, attribute_data, 10000)
images.shape

In [ ]:
[first_generated, labels], y = generate_real_samples([images, attributes], 9)
show_images(first_generated)

In [ ]:
[first_generated2, labels2], y2 = generate_fake_samples(generator, LATENT_DIM, 9, n_attributes=2)

first_generated2[0].max()
show_images(first_generated2)


In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = dataset[0].shape[0] // n_batch
    half_batch = n_batch // 2
    FILE_PATH = '%s/generated_%d.png'
    RES_DIR = 'resources'
    if not os.path.isdir(RES_DIR):
        os.mkdir(RES_DIR)

    print('STARTING TRAINING')
    # manually enumerate epochs
    CHANNELS = 3
    discriminator_loss = []
    gan_loss = []
    CONTROL_SIZE_SQRT = 3
    control_vectors = np.random.normal(size=(CONTROL_SIZE_SQRT**2, LATENT_DIM)) / 2
    control_image = np.zeros((WIDTH * CONTROL_SIZE_SQRT, HEIGHT * CONTROL_SIZE_SQRT, CHANNELS))
    control_generated = generator.predict(control_vectors)
    for i in range(CONTROL_SIZE_SQRT ** 2):
        x_off = i % CONTROL_SIZE_SQRT
        y_off = i // CONTROL_SIZE_SQRT
        control_image[x_off * WIDTH:(x_off + 1) * WIDTH, y_off * HEIGHT:(y_off + 1) * HEIGHT, :] = control_generated[i, :, :, :]  
    plt.imshow(control_image)
    plt.show()
    for epoch in range(n_epochs):
        temp1 = []
        temp2 = []
        print(bat_per_epo, dataset[0].shape[0], n_batch)
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_model.trainable = True
            y_real += .05 * np.random.random(y_real.shape)

            d_loss1 = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch, 2)
            y_fake += .05 * np.random.random(y_fake.shape)

            # update discriminator model weights
#             print(d_model.predict(X_fake).round(), y_fake)
            d_loss2= d_model.train_on_batch(X_fake, y_fake)
#             print(d_loss2, _)
            d_model.trainable = False

            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch, 2)
            # create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(z_input, y_gan)
            
            temp1.append(d_loss1 + d_loss2)
            temp2.append(g_loss)
            # summarize loss on this batch
            print('>Epoch:%d, Batch:%d/%d, Real=%.3f, Fake=%.3f GAN=%.3f' % (epoch+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss), end='\r')
        discriminator_loss.append(mean(temp1))
        gan_loss.append(mean(temp2))
#         [temp, labels], y = generate_fake_samples(generator, LATENT_DIM, 9, n_attributes=2)
#         show_images(temp)
        control_image = np.zeros((WIDTH * CONTROL_SIZE_SQRT, HEIGHT * CONTROL_SIZE_SQRT, CHANNELS))
        control_generated = generator.predict(control_vectors)
        for i in range(CONTROL_SIZE_SQRT ** 2):
            x_off = i % CONTROL_SIZE_SQRT
            y_off = i // CONTROL_SIZE_SQRT
            control_image[x_off * WIDTH:(x_off + 1) * WIDTH, y_off * HEIGHT:(y_off + 1) * HEIGHT, :] = control_generated[i, :, :, :]  
        plt.imshow(control_image)
        plt.show()
        im = Image.fromarray(np.uint8(control_image * 255))
        im.save(FILE_PATH % (RES_DIR, epoch))
        [X_fake, labels], y_fake = generate_fake_samples(generator, 100, 1, 2)
        print(discriminator.predict(X_fake))
    # save the generator model
    g_model.save('cgan_generator_base.h5')
    return discriminator_loss, gan_loss


In [ ]:
discriminator_loss, gan_loss = train(generator, discriminator, gan, [images, attributes], LATENT_DIM, 20, 32)


In [ ]:
[temp, labels], y = generate_fake_samples(generator, LATENT_DIM, 9, n_attributes=2)
show_images(temp)

In [ ]:
discriminator.predict(first_generated)
show_images(first_generated)

In [ ]:
[X_fake, labels], y_fake = generate_fake_samples(generator, 100, 1, 2)


In [ ]:
print(discriminator.predict([X_fake, labels]))

In [ ]:
plt.plot(discriminator_loss)
plt.show()

In [ ]:
plt.plot(gan_loss)
plt.show()

In [ ]:
import imageio
import shutil

RES_DIR = 'resources'

images_to_gif = []
for filename in os.listdir(RES_DIR):
    images_to_gif.append(imageio.imread(RES_DIR + '/' + filename))
imageio.mimsave('trainnig_visual.gif', images_to_gif)
shutil.rmtree(RES_DIR)